# ML Pipeline Preparation
For the machine learning portion, you will split the data into a training set and a test set. Then, you will create a machine learning pipeline that uses NLTK, as well as scikit-learn's Pipeline and GridSearchCV to output a final model that uses the message column to predict classifications for 36 categories (multi-output classification). Finally, you will export your model to a pickle file. After completing the notebook, you'll need to include your final machine learning code in train_classifier.py.

Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [61]:
# import libraries
import pandas as pd
pd.set_option('display.max_colwidth', 500)
from sqlalchemy import create_engine
import sqlite3
import re

In [62]:
import sqlite3
import pandas as pd

# connect to the database
conn = sqlite3.connect('../data/DisasterResponse.db')

# run a query
df = pd.read_sql('SELECT * FROM disaster_response_messages', conn)

In [63]:
df.shape

(26216, 40)

In [64]:
df.head(1)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that could pass over Haiti,Un front froid se retrouve sur Cuba ce matin. Il pourrait traverser Haiti demain. Des averses de pluie isolee sont encore prevues sur notre region ce soi,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [65]:
df.shape

(26216, 40)

In [66]:
df.columns[4:]

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [67]:
# def load_data():
#     df = pd.read_csv('corporate_messaging.csv', encoding='latin-1')
#     df = df[(df["category:confidence"] == 1) & (df['category'] != 'Exclude')]
#     X = df.text.values
#     y = df.category.values
#     return X, y

In [68]:
y = df[df.columns[4:]].values
X = df['message'].values

### 2. Write a tokenization function to process your text data

In [69]:
import nltk
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

nltk.download(['punkt', 'wordnet','stopwords'])

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/henriettewevell/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/henriettewevell/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/henriettewevell/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [70]:
# from nltk.tokenize import word_tokenize
# from nltk.corpus import stopwords
# # download necessary NLTK data
# import nltk
# nltk.download(['punkt', 'wordnet'])
# from nltk.stem.wordnet import WordNetLemmatizer

In [71]:
def replace_all_urls(text: str) -> str:
        url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
        detected_urls = re.findall(url_regex, text)
        for url in detected_urls:
             text = text.replace(url, "urlplaceholder")
        return text

def clean_text(text: str) -> str:
    text = text.lower()
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    test = text.strip() # remove whitespaces
    return test

def tokenize(text : str) -> list:
    words = word_tokenize(text)
    return words

def remove_stopwords(text : list) -> list:
    stop_words = stopwords.words("english")
    words = [w for w in text if w not in stop_words]
    return words

def lemmatize(text: list) -> list:
    lemmatized_all = []
    for word in text:
        lemmatizer = WordNetLemmatizer()
        lemmatized = lemmatizer.lemmatize(word, pos='v')
        lemmatized_all.append(lemmatized)
    return lemmatized_all


def clean_and_tokenize(text: str) -> list:
    text = replace_all_urls(text)
    text = clean_text(text)
    tokens = tokenize(text)
    tokens = remove_stopwords(tokens)
    tokens = lemmatize(tokens)
    return tokens

# def tokenize(text):
#     detected_urls = re.findall(url_regex, text)
#     for url in detected_urls:
#         text = text.replace(url, "urlplaceholder")

#     tokens = word_tokenize(text)
#     lemmatizer = WordNetLemmatizer()

#     clean_tokens = []
#     for tok in tokens:
#         clean_tok = lemmatizer.lemmatize(tok).lower().strip()
#         clean_tokens.append(clean_tok)

#     return clean_tokens


In [72]:
clean_and_tokenize("If you want to find a Job at an NGO or the Government, upload your resume at http://www.jobpaw.com/")

['want',
 'find',
 'job',
 'ngo',
 'government',
 'upload',
 'resume',
 'urlplaceholder']

In [73]:
# url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
# urls_df = df[df['message'].str.contains(url_regex)]['message'].head(1)
# urls_df

In [74]:
# dilter the df messages column for when url is present
# urls_df = df[df['message'].str.contains(url_regex)]['message'].head(5)
# urls_df
# number_regex = r'\d+,\d+'
# # or number.number
# number_regex2 = r'\d+\.\d+'
# # or number
# number_regex3 = r'\d+'

# df[df['message'].str.contains(number_regex2)]['message'].sample(n=15)

# filter the df messages column for number_regex or number_regex2 or number_regex3 is present
# df[df['message'].str.contains(number_regex) | df['message'].str.contains(number_regex2) | df['message'].str.contains(number_regex3)]['message'].sample(n=15)

In [75]:
# tokenize(df.iloc[10]['message'])

In [76]:
# number_and_letters_regex = r'\d+[a-zA-Z]+'
# letters_and_number_regex = r'[a-zA-Z]+\d+'
# combined_regex = r'(\d+[a-zA-Z]+)|([a-zA-Z]+\d+)'
# # df[df['message'].str.contains(number_and_letters_regex)]['message'].sample(n=15)

# # get number_and_letters_regex from each message
# df[df['message'].str.contains(letters_and_number_regex)]['message'].str.findall(letters_and_number_regex).head()

In [77]:
# df[df['message'].str.contains(number_and_letters_regex)]['message'].str.findall(number_and_letters_regex).head()

In [78]:
# df[df['message'].str.contains(combined_regex)]['message'].str.findall(combined_regex).head()

In [79]:
# filter df messages for when it contains any numbers
# df[df['message'].str.contains('\d')]['message'].sample(n=10)

In [80]:
# remove stopwords
# lower case
# lemmatize?
# remove urls
# remove punctuation
# remove numbers?

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [81]:
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [82]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [83]:
from sklearn.model_selection import train_test_split
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [84]:
# Train pipeline
clf_model = pipeline.fit(X_train, y_train)

/Users/henriettewevell/src/udacity_disaster_response/venv/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [85]:
def main():
    # load data and perform train text split
    X, y = load_data()
    X_train, X_test, y_train, y_test = train_test_split(X, y)

    # instantiate transformers and classifiers
    vect = CountVectorizer(tokenizer=tokenize)
    tfidf = TfidfTransformer()
    clf = RandomForestClassifier()

    # fit and transform the training data
    X_train_counts = vect.fit_transform(X_train)
    X_train_tfidf = tfidf.fit_transform(X_train_counts)

    # train classifier
    clf.fit(X_train_tfidf, y_train)

    # transform (no fitting) the test data
    X_test_counts = vect.transform(X_test)
    X_test_tfidf = tfidf.transform(X_test_counts)
    # predict on test data
    y_pred = clf.predict(X_test_tfidf)

    # display results
    display_results(y_test, y_pred)

In [86]:
import numpy as np
from sklearn.metrics import confusion_matrix
def display_results(y_test, y_pred):
    labels = np.unique(y_pred)
    confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    accuracy = (y_pred == y_test).mean()

    print("Labels:", labels)
    print("Confusion Matrix:\n", confusion_mat)
    print("Accuracy:", accuracy)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [87]:
# predict on test data
y_pred = clf_model.predict(X_test)

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [88]:
# display results
# display_results(y_test, y_pred) 

In [104]:
y_columns = df[df.columns[4:]].columns
y_columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [122]:
from sklearn.metrics import classification_report
for cols in y_columns:
    print(f"Category: {cols}\n"
          , classification_report(y_test[:,y_columns.get_loc(cols)], y_pred[:,y_columns.get_loc(cols)]))

Category: related
               precision    recall  f1-score   support

           0       0.73      0.26      0.39      1266
           1       0.80      0.97      0.87      3938
           2       0.75      0.07      0.14        40

    accuracy                           0.79      5244
   macro avg       0.76      0.44      0.47      5244
weighted avg       0.78      0.79      0.75      5244

Category: request
               precision    recall  f1-score   support

           0       0.89      0.99      0.94      4349
           1       0.89      0.41      0.56       895

    accuracy                           0.89      5244
   macro avg       0.89      0.70      0.75      5244
weighted avg       0.89      0.89      0.87      5244

Category: offer
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      5218
           1       0.00      0.00      0.00        26

    accuracy                           1.00      5244
   macro avg       

/Users/henriettewevell/src/udacity_disaster_response/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/henriettewevell/src/udacity_disaster_response/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/henriettewevell/src/udacity_disaster_response/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to c

Category: infrastructure_related
               precision    recall  f1-score   support

           0       0.94      1.00      0.97      4916
           1       0.00      0.00      0.00       328

    accuracy                           0.94      5244
   macro avg       0.47      0.50      0.48      5244
weighted avg       0.88      0.94      0.91      5244

Category: transport
               precision    recall  f1-score   support

           0       0.96      1.00      0.98      5004
           1       0.77      0.08      0.15       240

    accuracy                           0.96      5244
   macro avg       0.86      0.54      0.56      5244
weighted avg       0.95      0.96      0.94      5244

Category: buildings
               precision    recall  f1-score   support

           0       0.95      1.00      0.98      4977
           1       0.86      0.07      0.12       267

    accuracy                           0.95      5244
   macro avg       0.90      0.53      0.55      524

/Users/henriettewevell/src/udacity_disaster_response/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/henriettewevell/src/udacity_disaster_response/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/henriettewevell/src/udacity_disaster_response/venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to c

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
# use grid search to find better parameters
from sklearn.model_selection import GridSearchCV

parameters = {
    'vect__ngram_range': ((1, 1), (1, 2)),
    'tfidf__use_idf': (True, False),
    'clf__estimator__n_estimators': [50, 100, 200],
    'clf__estimator__min_samples_split': [2, 3, 4],
}

cv = GridSearchCV(pipeline, param_grid=parameters, verbose=3)


In [94]:
parameters = 

cv = 

SyntaxError: invalid syntax (3277122076.py, line 1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [95]:
import pickle
pickle.dump(clf_model, open('model.pkl', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.

In [96]:
from sklearn.datasets import make_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# generate some random data for demonstration purposes
X, y = make_classification(n_samples=100, n_features=5, n_classes=3, n_informative=3, random_state=42)

# create a MultiOutputClassifier with a RandomForestClassifier as the base estimator
multi_target_rf = MultiOutputClassifier(RandomForestClassifier(random_state=42))

# fit the MultiOutputClassifier on the data
multi_target_rf.fit(X, y)




ValueError: y must have at least two dimensions for multi-output regression but has only one.

In [ ]:
# iterate over each output category (i.e. each column in the target variable)
for i in range(y.shape[1]):
    target_col = y[:, i]
    predicted_col = multi_target_rf.predict(X)[:, i]
    print(f"Classification report for output category {i}:")
    print(classification_report(target_col, predicted_col))